In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi
import os

file_path = 'school_data.csv'  
data = pd.read_csv(file_path)

# Ensure 'Player' column exists and handle case sensitivity
data.columns = data.columns.str.strip().str.lower()

output_dir = 'player_radar_charts'
os.makedirs(output_dir, exist_ok=True)

# Function to normalize data between 0 and 1
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

# Columns for each of the five matrices
categories = {
    'Attacking Contribution': ['goals', 'assists', 'shots_on_target', 'xa', 'progressive_runs_attempted', 'crosses', 'accurate_crosses'],
    'Defensive Capability': ['defensive_duels', 'slide_tackles', 'interceptions', 'recoveries', 'recoveries_in_own_half', 'loose_ball_duels'],
    'Passing & Playmaking': ['total_passes', 'forward_passes', 'passes_into_final_third', 'passes_into_box', 'shot_assists', 'second_assists'],
    'Duels & Physical Presence': ['total_duels', 'aerial_duels', 'offensive_duels', 'fouls_drawn', 'touches_inside_box'],
    'Discipline & Goalkeeping': ['fouls', 'yellow_cards', 'red_cards', 'gk_stat_saves', 'gk_stat_reflex_saves']
}

# Generate radar chart for each player in the dataset
def radar_chart(data):
    # Ensure 'player' column exists in lowercase
    if 'player_name' not in data.columns:
        print("The dataset does not contain a 'Player' column.")
        return

    sns.set(style="whitegrid")  # Set seaborn style for better aesthetics

    for player_name in data['player_name'].unique():
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})

        # Preparing data for each category
        values = []
        baseline_values = []
        labels = []
        for category, metrics in categories.items():
            available_metrics = [metric for metric in metrics if metric in data.columns]
            category_values = normalize(data[available_metrics].sum(axis=1))
            baseline_value = category_values.mean()
            
            # Player specific values
            player_values = category_values[data['player_name'] == player_name].values[0]
            
            values.append(player_values * 10)  # Scale value to 0-10
            baseline_values.append(baseline_value * 10)  # Scale baseline to 0-10
            labels.append(category)

        # Ensure radar values are on the same scale (0 to 10)
        values = [min(10, max(0, v)) for v in values]  # Cap values at 10
        baseline_values = [min(10, max(0, v)) for v in baseline_values]  # Cap values at 10

        # Radar chart setup
        values += values[:1]  # Close the radar chart circle
        baseline_values += baseline_values[:1]  # Close the radar chart circle
        
        N = len(labels)
        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]

        ax.set_theta_offset(pi / 2)
        ax.set_theta_direction(-1)

        plt.xticks(angles[:-1], labels, fontsize=12, color='navy')
        ax.plot(angles, baseline_values, linewidth=2, linestyle='dotted', label='Baseline (Average)', color='grey')
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=player_name, color='blue')
        ax.fill(angles, values, color='blue', alpha=0.3)

        # Annotate each value on the radar chart
        for i, value in enumerate(values[:-1]):
            ax.text(angles[i], value + 0.5, f'{value:.1f}', horizontalalignment='center', size=10, color='darkred', weight='semibold')

        ax.set_ylim(0, 10)  # Set the same scale for all radars with max 10
        
        plt.title(f'{player_name} Performance Radar Chart', size=15, color='darkblue', weight='bold', pad=20)
        plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

        # Save the figure to the output directory
        output_path = os.path.join(output_dir, f'{player_name}.png')
        plt.savefig(output_path)
        plt.close()

radar_chart(data)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
